In [1]:
import flowshape as fs
import igl
import numpy as np
import meshplot as mp
import os
from src.utilities.fin_shape_utils import fit_fin_hull, upsample_fin_point_cloud, plot_mesh
from src.utilities.fin_class_def import FinData
from src.utilities.functions import path_leaf
import glob2 as glob

### Load fin data

In [2]:
# root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"
root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"
fin_object_path = os.path.join(root, "point_cloud_data", "fin_objects", "")
fin_object_list = sorted(glob.glob(fin_object_path + "*.pkl"))

file_ind01 = 146
seg_type = "tissue_only_best_model_tissue"
fp01 = fin_object_list[file_ind01]
point_prefix01 = path_leaf(fp01).replace("_fin_object.pkl", "")
print(point_prefix01)

fin_object = FinData(data_root=root, name=point_prefix01, tissue_seg_model=seg_type)

20240711_02_well0053_time0000


### Get fin mesh

In [3]:
# sample nucleus boundary points from nucleus masks
fin_df_upsamp = upsample_fin_point_cloud(fin_object, sample_res_um=0.4, root=root, points_per_nucleus=100)

/home/nick/projects/pecfin_dynamics/src/utilities/fin_shape_utils.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_df["nn_scale_um"] = nn_dist_mean
100%|██████████| 597/597 [00:01<00:00, 376.20it/s]


In [4]:
import open3d as o3d

# get raw points
fin_points = fin_df_upsamp[["XP", "YP", "ZP"]].to_numpy()

# convert to point cloud format
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(fin_points)

# resample points to be more spatially uniform
min_distance = 0.5
print("Downsampling...")
sampled_points = pcd.voxel_down_sample(voxel_size=min_distance) 
print("Done")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Downsampling...
Done


In [5]:
# import plotly.express as px
# fin_points_u = np.asarray(sampled_points.points)

# fig = px.scatter_3d(x=fin_points_u[:, 0], y=fin_points_u[:, 1], z=fin_points_u[:, 2])
# fig.update_traces(marker=dict(size=3))
# fig.show()

In [7]:
# fit a mesh 
fin_points_u = np.asarray(sampled_points.points)
fin_hull, raw_hull, wt_flag = fit_fin_hull(fin_points_u, alpha=25, n_faces=5000)
print(wt_flag)

True


In [39]:
raw_hull.faces.shape

(10118, 3)

In [8]:
#igl.read_triangle_mesh("/home/nick/projects/flowshape/demo/ABal.obj")
v, f = v, f = fin_hull.vertices.copy(), fin_hull.faces.copy()
mp.plot(v, f, shading = {"wireframe":True})

/home/nick/miniconda3/envs/flowshape_env/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(
/home/nick/miniconda3/envs/flowshape_env/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.368526…

### Run spherical mapping

In [9]:
# normalize the scaling of the mesh
v = fs.normalize(v)

# run the spherical mapping flow and mobius centering
sv = fs.sphere_map(v, f)

# Now we have a spherical mesh
mp.plot(sv, f, shading = {"wireframe":True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000453…

### Calculate the mean curvature

In [10]:
rho = fs.curvature_function(v, sv, f)

mp.plot(v,f, rho )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0033084…

## SH decomposition

In [11]:
# this utility does the above steps + SH decomposition
# Here, using maximum degree 24
weights, Y_mat, vs = fs.do_mapping(v,f, l_max = 24)

In [12]:
# This is the array of SH weights
np.set_printoptions(threshold = 100)
print(weights)

[ 3.1985965  -0.14189246 -0.09633128 ...  0.18525062 -0.10537151
  0.13012568]


In [13]:
# Y_mat is a matrix used to convert between weights and mesh function
rho2 = Y_mat.dot(weights)
mp.plot(sv,f, c = rho2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000453…

### Mesh reconstruction

In [14]:
rec2 = fs.reconstruct_shape(sv, f, rho2 )
mp.plot(rec2,f, c = rho2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.050321…

### Test with lower frequencies only

In [18]:
weights, Y_mat, vs = fs.do_mapping(v,f, l_max=4)
rec_8 = fs.reconstruct_shape(sv, f, Y_mat.dot(weights) )
mp.plot(rec_8, f, c = rho2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.078880…